In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam
from keras.losses import MeanSquaredError, BinaryCrossentropy

import optuna

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [3]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [5]:
params = {
    'hidden_layer_units': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': [0.0001, 0.01],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'output_activation': ['sigmoid', 'softmax'],
    'loss_function': ['binary_crossentropy', 'mse'],
    'dropout_rate': [0.5, 0.3, 0.02]
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    input_dimension = X_train.shape[1]
    
    model.add(
        Dense(
            input_dim=input_dimension,
            units=trial.suggest_categorical(
                'hidden_layer_units',
                params['hidden_layer_units']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        )
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                units=trial.suggest_categorical(
                    'hidden_layer_units',
                    params['hidden_layer_units']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )

        model.add(
            Dropout(
                trial.suggest_categorical(
                    'dropout_rate',
                    params['dropout_rate']
                )
            )
        )
    
    model.add(
        Dense(
            units=1,
            activation=trial.suggest_categorical(
                'output_activation',
                params['output_activation']
            )
        )
    )

    alpha = trial.suggest_categorical('alpha', params['alpha'])
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss=trial.suggest_categorical('loss_function', params['loss_function']),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [6]:
n_trials = 10

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-03-31 00:36:55,804] A new study created in memory with name: no-name-5ac79d01-cbbe-465e-b3bd-1db68f16a0d8


Epoch 1/10000
3953/3953 [==============================] - 9s 2ms/step - loss: 0.6334 - accuracy: 0.6487 - val_loss: 0.6077 - val_accuracy: 0.6681
Epoch 2/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6110 - accuracy: 0.6657 - val_loss: 0.6034 - val_accuracy: 0.6710
Epoch 3/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6061 - accuracy: 0.6707 - val_loss: 0.6013 - val_accuracy: 0.6724
Epoch 4/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6039 - accuracy: 0.6723 - val_loss: 0.6004 - val_accuracy: 0.6740
Epoch 5/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6020 - accuracy: 0.6735 - val_loss: 0.6007 - val_accuracy: 0.6726
Epoch 6/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6008 - accuracy: 0.6751 - val_loss: 0.5991 - val_accuracy: 0.6746
Epoch 7/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.5998 - accuracy: 0.6761 - val_loss: 0

[I 2023-03-31 00:42:04,994] Trial 0 finished with value: 0.5909279584884644 and parameters: {'hidden_layer_units': 32, 'activation': 'tanh', 'hidden_layers': 1, 'dropout_rate': 0.5, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'loss_function': 'binary_crossentropy', 'batch_size': 64, 'max_iter': 10000}. Best is trial 0 with value: 0.5909279584884644.


Epoch 1/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6761 - accuracy: 0.5915 - val_loss: 0.6543 - val_accuracy: 0.6478
Epoch 2/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6532 - accuracy: 0.6486 - val_loss: 0.6508 - val_accuracy: 0.6501
Epoch 3/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6501 - accuracy: 0.6499 - val_loss: 0.6480 - val_accuracy: 0.6511
Epoch 4/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6474 - accuracy: 0.6508 - val_loss: 0.6455 - val_accuracy: 0.6513
Epoch 5/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6448 - accuracy: 0.6512 - val_loss: 0.6433 - val_accuracy: 0.6518
Epoch 6/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6428 - accuracy: 0.6517 - val_loss: 0.6413 - val_accuracy: 0.6524
Epoch 7/5000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6408 - accuracy: 0.6520 - val_loss: 0.6395 -

[I 2023-03-31 00:51:35,154] Trial 1 finished with value: 0.6108361482620239 and parameters: {'hidden_layer_units': 128, 'activation': 'tanh', 'hidden_layers': 1, 'dropout_rate': 0.02, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'loss_function': 'binary_crossentropy', 'batch_size': 128, 'max_iter': 5000}. Best is trial 0 with value: 0.5909279584884644.


...

[I 2023-03-31 01:51:23,357] Trial 7 finished with value: 0.2043662667274475 and parameters: {'hidden_layer_units': 32, 'activation': 'sigmoid', 'hidden_layers': 1, 'dropout_rate': 0.02, 'output_activation': 'sigmoid', 'alpha': 0.01, 'optimizer': 'Adam', 'loss_function': 'mse', 'batch_size': 64, 'max_iter': 5000}. Best is trial 7 with value: 0.2043662667274475.


Epoch 1/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.6049 - accuracy: 0.6555 - val_loss: 0.5981 - val_accuracy: 0.6554
Epoch 2/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5965 - accuracy: 0.6555 - val_loss: 0.5986 - val_accuracy: 0.6554
Epoch 3/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5953 - accuracy: 0.6555 - val_loss: 0.5958 - val_accuracy: 0.6554
Epoch 4/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5935 - accuracy: 0.6555 - val_loss: 0.5957 - val_accuracy: 0.6554
Epoch 5/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5929 - accuracy: 0.6555 - val_loss: 0.5952 - val_accuracy: 0.6554
Epoch 6/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5922 - accuracy: 0.6555 - val_loss: 0.5943 - val_accuracy: 0.6554
Epoch 7/2000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.5916 - accuracy: 0.6555 - val_loss: 0

[I 2023-03-31 01:54:44,219] Trial 8 finished with value: 0.5930061936378479 and parameters: {'hidden_layer_units': 32, 'activation': 'sigmoid', 'hidden_layers': 2, 'dropout_rate': 0.02, 'output_activation': 'softmax', 'alpha': 0.01, 'optimizer': 'Adam', 'loss_function': 'binary_crossentropy', 'batch_size': 64, 'max_iter': 2000}. Best is trial 7 with value: 0.2043662667274475.


Epoch 1/5000
7906/7906 [==============================] - 23s 3ms/step - loss: 0.6511 - accuracy: 0.6506 - val_loss: 0.6434 - val_accuracy: 0.6554
Epoch 2/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6450 - accuracy: 0.6555 - val_loss: 0.6449 - val_accuracy: 0.6554
Epoch 3/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6445 - accuracy: 0.6555 - val_loss: 0.6431 - val_accuracy: 0.6554
Epoch 4/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6443 - accuracy: 0.6555 - val_loss: 0.6427 - val_accuracy: 0.6554
Epoch 5/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6435 - accuracy: 0.6555 - val_loss: 0.6420 - val_accuracy: 0.6554
Epoch 6/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6425 - accuracy: 0.6554 - val_loss: 0.6393 - val_accuracy: 0.6554
Epoch 7/5000
7906/7906 [==============================] - 22s 3ms/step - loss: 0.6413 - accuracy: 0.6554 - val_loss: 0

[I 2023-03-31 02:15:55,773] Trial 9 finished with value: 0.6017345190048218 and parameters: {'hidden_layer_units': 128, 'activation': 'sigmoid', 'hidden_layers': 1, 'dropout_rate': 0.5, 'output_activation': 'sigmoid', 'alpha': 0.01, 'optimizer': 'SGD', 'loss_function': 'binary_crossentropy', 'batch_size': 32, 'max_iter': 5000}. Best is trial 7 with value: 0.2043662667274475.


In [7]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

Best hyperparameters:  {'hidden_layer_units': 32, 'activation': 'sigmoid', 'hidden_layers': 1, 'dropout_rate': 0.02, 'output_activation': 'sigmoid', 'alpha': 0.01, 'optimizer': 'Adam', 'loss_function': 'mse', 'batch_size': 64, 'max_iter': 5000}
Best objective value:  0.2043662667274475


#### **Saving study**

In [8]:
import joblib

In [9]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']